# Diabetic patients readmission rates preditction


In [ ]:
pip install tqdm

In [ ]:
pip install learn2learn

In [ ]:
pip install tensorflow

In [3]:
# Importing packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import optimize
from sklearn import datasets as skdataset
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler


import os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.utils.data import Dataset


from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam


from tqdm import tqdm

In [4]:
from learn2learn.algorithms.maml import MAML
from learn2learn.data import TaskDataset

In [5]:
pd.set_option('display.max_columns', None)

# Project introduction

- Overview: <br>
This project is focusing on developing a predictive model to ascertain the likelihood of readmission for diabetes patients.
<br>

- Target:<br>
The main goal of this project is developing a powerful machine learning model which can predict the readmission rate of patient 

# Data loading
The following cells are used to load training and testing data for our prediction

In [6]:
train_data = pd.read_csv("Dataset/diabetic_data_training.csv")
test_data = pd.read_csv("Dataset/diabetic_data_test.csv")
mapping_info = pd.read_csv("Dataset/IDS_mapping.csv", header=None)

# Data Visualization

In [7]:
# # Function to handle different data types for plotting
# def plot_column(ax, column, df):
#     if df[column].dtype == 'object':
#         # Check if binary
#         if df[column].nunique() == 2:
#             # Binary data visualization
#             df[column].value_counts().plot(kind='bar', ax=ax)
#         else:
#             # Categorical data visualization
#             df[column].value_counts().plot(kind='pie', autopct='%1.1f%%', ax=ax)
#     elif df[column].dtype == 'int64' or df[column].dtype == 'float64':
#         # Numeric data visualization
#         df[column].plot(kind='hist', bins=20, ax=ax)
#     else:
#         ax.text(0.5, 0.5, f"Unhandled data type for column: {column}", 
#                 fontsize=12, ha='center')
#     ax.set_title(column)

# # Creating a 4x4 subplot layout
# fig, axes = plt.subplots(nrows=13, ncols=4, figsize=(20, 65))
# fig.tight_layout(pad=5.0)

# # Iterate through each column and plot
# for i, col in enumerate(train_data.columns):
#     # Adjust this line to select different subsets of columns  
#     plot_column(axes[i//4, i%4], col,train_data)


# plt.show()


# Data preprocessing
The following cells are used to preprocess the training and testing data. There are two main goals in our preprocessing data section of the code
- Change the string type data in our dataset to integer type data 
- Apply some applicable method to full up the missing value

In [8]:
# # this part just make a dictionary to mapping the id between data and ID mapping table
# mapping_dicts = [{} for i in range(3)]
# mapping_title = []
# title = -1
# for data in mapping_info.itertuples():
#     if(pd.isna(data[1])):
#         continue
#     if(not data[1].isdigit()):
#         title += 1
#         mapping_title.append(data[1])
#     else:
#         if(pd.isna(data[2])):
#             mapping_dicts[title]['NULL'] = data[1]
#             continue
#         mapping_dicts[title][data[2]] = data[1]

In [9]:
# this part is used to change all string type data to integer type
# for the missing value, we will skip and process it at next step
df = train_data.copy()


df_test = test_data.copy()


In [10]:

# mask = df.isna().sum(axis=1) <= 3
# filtered_df = df[mask]
# print(filtered_df.any(axis=1).sum())

One-Hot Encoding For race:
cons: One-hot encoding can lead to a significant increase in the dataset's dimensionality (a problem known as the "curse of dimensionality"), especially if the categorical feature has many unique values. This can increase the computational cost and may require more data to achieve good performance.
Dems Redct Would be apply, so it doesn't matter
pros: Map to a fix number implies an ordinal relationship between the categories which may not exist, but is ideal for non-ordinal categorical data. It's suitable for many machine learning models, especially those that assume no ordinal relationship between categories


1. random forest, remove ?
2. randomly assign ? to a class by disstribution

General Missing value
1. multiple imputation To be decide when training if less than 1h 5 epoch
2. mean
3. fullly remove
4. wrong -> fix ?


Encoding for age:
1. Asumming normal distribution, map to a random age in the range
2. Map to mean age in the range

Race Process
1. Remove missing since race is proved to be a significant impact to medical result.
2. One-hot encode race

In [11]:
def race_filter(df):
    """
    Remove missing value from race
    
    Parameters:
    df (pandas.dataframe): series containing a colum of the feature matrix.

    Return:
    Filtered dataframe free from ? for race
    """
    race_mask = (df['race'] != "?")
    df = df[race_mask]
    return df

In [12]:
df = race_filter(df)
df_encoded = pd.get_dummies(df, columns=["race"], prefix="race",dtype=int)

df_test = race_filter(df_test)
df_test_encoded = pd.get_dummies(df_test, columns=["race"], prefix="race",dtype=int)

Gender Process
1. Remove Unknown/Invalid and missing
2. One hot encode Gender

In [13]:
def gender_filter(df):
    """
    Remove missing value from race
    
    Parameters:
    df (pandas.dataframe): series containing a colum of the feature matrix.

    Return:
    Filtered dataframe free from ? for race
    """
    gender_mask = (df['gender'] != "Unknown/Invalid")
    df = df[gender_mask]
    return df

In [14]:
df_encoded = gender_filter(df_encoded)
df_test_encoded = gender_filter(df_test_encoded)

In [15]:
gender_mapping = {'Male':0,'Female':1}
df_encoded['gender'] = df_encoded['gender'].map(gender_mapping)
df_test_encoded['gender'] = df_test_encoded['gender'].map(gender_mapping)

encounter_id and patient_nbr Process 
1. n/a
2. Drop since it provide no info to the result

In [16]:
df_encoded.drop(columns=['encounter_id','patient_nbr'], inplace=True)
df_test_encoded.drop(columns=['encounter_id','patient_nbr'],inplace=True)

Age Process
1. n/a
2. Map age from range to mean

In [17]:
df_encoded['age'] = (df['age'].str.extract(r'(\d+)-(\d+)')[0].astype(int)+df['age'].str.extract(r'(\d+)-(\d+)')[1].astype(int))//2
df_test_encoded['age'] = (df_test['age'].str.extract(r'(\d+)-(\d+)')[0].astype(int)+df_test['age'].str.extract(r'(\d+)-(\d+)')[1].astype(int))//2

Weight Process
1. Drop Weight for too many missing values and no information to predict.
2. n/a

In [18]:
df_encoded.drop(columns=['weight'], inplace=True)
df_test_encoded.drop(columns=['weight'], inplace=True)

admission_type_id
discharge_disposition_id
admission_source_id
To be merged

Payer Code and Medical Specialty Process
1. Both droped since too many missing
2. Payer Code not relevant as id, Medical Specialty too many categories and no info to predict.

In [19]:
df_encoded.drop(columns=['payer_code'], inplace=True)
df_encoded.drop(columns=['medical_specialty'], inplace=True)
df_test_encoded.drop(columns=['payer_code'], inplace=True)
df_test_encoded.drop(columns=['medical_specialty'], inplace=True)

In [20]:
# df_pc = df['payer_code'].astype('category')
# unique_categories_pc = df_pc.cat.categories
# medical_specialty_to_num = {medical_specialty: i for i, medical_specialty in enumerate(unique_categories_pc)}
# max_binary_length = len(bin(len(unique_categories_pc) - 1)) - 2
# payer_code_to_binary = {medical_specialty: bin(num)[2:].zfill(max_binary_length) for medical_specialty, num in medical_specialty_to_num.items()}

# df_encoded['payer_code'] = df_encoded['payer_code'].map(payer_code_to_binary)

# df_test_encoded['payer_code'] = df_test_encoded['payer_code'].map(payer_code_to_binary)
# df_ms = df['medical_specialty'].astype('category')
# unique_categories_ms = df_ms.cat.categories
# medical_specialty_to_num = {medical_specialty: i for i, medical_specialty in enumerate(unique_categories_ms)}
# max_binary_length = len(bin(len(unique_categories_ms) - 1)) - 2
# medical_specialty_to_binary = {medical_specialty: bin(num)[2:].zfill(max_binary_length) for medical_specialty, num in medical_specialty_to_num.items()}

# df_encoded['medical_specialty'] = df_encoded['medical_specialty'].map(medical_specialty_to_binary)

# df_test_encoded['medical_specialty'] = df_test_encoded['medical_specialty'].map(medical_specialty_to_binary)

num_lab_procedures
num_procedures
num_medications
number_outpatient
number_emergency
number_inpatient
No need to be encode since no missing and integer.

diag_1,diag_2 and diag_3 Process:
1. Filte diag_1, diag_2 and diag_3 by number_diagnoses. If the amount of diagnoses in diag_1, diag_2 and diag_3 doesn't match the number_diagnoses, then remove.
2. Layer encoding diags according to the ICD-9 Code Category, drop original 3 diag

In [21]:
# # Define the encoding function
# def encode_element(element):
#     encoding_result = [0] * 20  # Default encoding

#     if pd.notna(element):
#         if element[0].isdigit():
#             element = float(element)
#             # Encode based on numeric range
#             if 1 <= element <= 139:
#                 encoding_result[0] = 1
#             elif 140 <= element <= 239:
#                 encoding_result[1] = 1
#             elif 240 <= element <= 279:
#                 encoding_result[2] = 1
#             elif 280 <= element <= 289:
#                 encoding_result[3] = 1
#             elif 290 <= element <= 319:
#                 encoding_result[4] = 1
#             elif 320 <= element <= 389:
#                 encoding_result[5] = 1
#             elif 390 <= element <= 459:
#                 encoding_result[6] = 1
#             elif 460 <= element <= 519:
#                 encoding_result[7] = 1
#             elif 520 <= element <= 579:
#                 encoding_result[8] = 1
#             elif 580 <= element <= 629:
#                 encoding_result[9] = 1
#             elif 630 <= element <= 679:
#                 encoding_result[10] = 1
#             elif 680 <= element <= 709:
#                 encoding_result[11] = 1
#             elif 710 <= element <= 739:
#                 encoding_result[12] = 1
#             elif 740 <= element <= 759:
#                 encoding_result[13] = 1
#             elif 760 <= element <= 779:
#                 encoding_result[14] = 1
#             elif 780 <= element <= 799:
#                 encoding_result[15] = 1
#             elif 800 <= element <= 999:
#                 encoding_result[16] = 1
#             # Add more conditions for other ranges if needed
#         elif element[0].isalpha():
#             # Encode based on string prefix
#             if element.startswith('E'):
#                 encoding_result[17] = 1
#             elif element.startswith('V'):
#                 encoding_result[18] = 1
#             elif element.startswith('M'):
#                 encoding_result[19] = 1
#             # Add more conditions for other prefixes if needed

#     return encoding_result

In [22]:
# a = df["diag_1"].loc[0]
# a = encode_element(a)
# a

In [23]:
def diag_missing_value_filter(df):

    df['number_diagnoses'] = pd.to_numeric(df['number_diagnoses'], errors='coerce')
    mask = ((df['number_diagnoses'] <= 3) & (
            (3 - df[['diag_1', 'diag_2', 'diag_3']].apply(lambda x: x == "?").sum(axis=1)) >= df['number_diagnoses'])) | (
    (df['number_diagnoses'] > 3) & (df[['diag_1', 'diag_2', 'diag_3']].apply(lambda x: x == "?").sum(axis=1)) == 0) 
    df_filtered = df[mask].copy()

    return df_filtered

def diag_layer_encoding(df):

    # Define the encoding function
    def encode_element(element):
        encoding_result = [0] * 20  # Default encoding

        if pd.notna(element):
            if element[0].isdigit():
                element = float(element)
                # Encode based on numeric range
                if 1 <= element <= 139:
                    encoding_result[0] = 1
                elif 140 <= element <= 239:
                    encoding_result[1] = 1
                elif 240 <= element <= 279:
                    encoding_result[2] = 1
                elif 280 <= element <= 289:
                    encoding_result[3] = 1
                elif 290 <= element <= 319:
                    encoding_result[4] = 1
                elif 320 <= element <= 389:
                    encoding_result[5] = 1
                elif 390 <= element <= 459:
                    encoding_result[6] = 1
                elif 460 <= element <= 519:
                    encoding_result[7] = 1
                elif 520 <= element <= 579:
                    encoding_result[8] = 1
                elif 580 <= element <= 629:
                    encoding_result[9] = 1
                elif 630 <= element <= 679:
                    encoding_result[10] = 1
                elif 680 <= element <= 709:
                    encoding_result[11] = 1
                elif 710 <= element <= 739:
                    encoding_result[12] = 1
                elif 740 <= element <= 759:
                    encoding_result[13] = 1
                elif 760 <= element <= 779:
                    encoding_result[14] = 1
                elif 780 <= element <= 799:
                    encoding_result[15] = 1
                elif 800 <= element <= 999:
                    encoding_result[16] = 1
                # Add more conditions for other ranges if needed
            elif element[0].isalpha():
                # Encode based on string prefix
                if element.startswith('E'):
                    encoding_result[17] = 1
                elif element.startswith('V'):
                    encoding_result[18] = 1
                elif element.startswith('M'):
                    encoding_result[19] = 1
                # Add more conditions for other prefixes if needed

        return encoding_result

    for i in range(1,4):
        encoded_columns = df["diag_"+str(i)].apply(encode_element)

    # Create new columns with "diag_1" as a prefix
        for j in range(20):
            new_column_name = "diag_" + str(i)+"_"+ str(j+1)
            df[new_column_name] = encoded_columns.apply(lambda x: x[j])

    return df
# def diag_layer_encoding(element):
    if isinstance(element, (int,float)):
        # Encode based on numeric range
        if 1 <= element <= 139:
            return [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        elif 140 <= element <= 239:
            return [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        elif 240 <= element <= 279:
            return [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        elif 280 <= element <= 289:
            return [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        elif 290 <= element <= 319:
            return [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        elif 320 <= element <= 389:
            return [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        elif 390 <= element <= 459:
            return [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        elif 460 <= element <= 519:
            return [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        elif 520 <= element <= 579:
            return [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        elif 580 <= element <= 629:
            return [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
        elif 630 <= element <= 679:
            return [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
        elif 680 <= element <= 709:
            return [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
        elif 710 <= element <= 739:
            return [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
        elif 740 <= element <= 759:
            return [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
        elif 760 <= element <= 779:
            return [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
        elif 780 <= element <= 799:
            return [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
        elif 800 <= element <= 999:
            return [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
        # Add more conditions for other ranges if needed

    elif isinstance(element, str):
        # Encode based on string prefix
        if element.startswith('E'):
            return [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
        elif element.startswith('V'):
            return [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
        elif element.startswith('M'):
            return [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
        # Add more conditions for other prefixes if needed

    # Default case (not a number or string with known prefix)
    return [0] * 20

In [24]:
df_encoded = diag_missing_value_filter(df_encoded)
df_encoded = diag_layer_encoding(df_encoded)
df_test_encoded = diag_missing_value_filter(df_test_encoded)
df_test_encoded = diag_layer_encoding(df_test_encoded)

for i in range(1,4):
        df_encoded.drop(columns=[f'diag_{i}'], inplace=True)
        df_test_encoded.drop(columns=[f'diag_{i}'], inplace=True)


number_diagnoses pass since no missing and integer
max_glu_serum and A1Cresult Process:
1.n/a
2.index map to 0-3

In [25]:
max_glu_serum_mapping = {'>200': 2, '>300': 3, 'normal': 1}
A1Cresult_mapping = {'>8':3,'>7':2,'normal':1}

In [26]:
df_encoded['max_glu_serum'] = df_encoded['max_glu_serum'].map(max_glu_serum_mapping).fillna(0)
df_test_encoded['max_glu_serum'] = df_test_encoded['max_glu_serum'].map(max_glu_serum_mapping).fillna(0)

df_encoded["A1Cresult"] = df_encoded['A1Cresult'].map(A1Cresult_mapping).fillna(0)
df_test_encoded["A1Cresult"] = df_test_encoded['A1Cresult'].map(A1Cresult_mapping).fillna(0)

metformin
repaglinide
nateglinide
chlorpropamide
glimepiride
acetohexamide
glipizide
glyburide
tolbutamide
pioglitazone
rosiglitazone
acarbose
miglitol
troglitazone
tolazamide
examide
citoglipton
insulin
glyburide-metformin
glipizide-metformin
glimepiride-pioglitazone
metformin-rosiglitazone
metformin-pioglitazone
Process:
1. drop 'examide', 'citoglipton','glimepiride-pioglitazone' and 'metformin-rosiglitazone' since single value observed
2. index map to 0-3

In [27]:
df_encoded.drop(columns=['examide', 'citoglipton','glimepiride-pioglitazone','metformin-rosiglitazone'],axis=1,inplace=True)
df_test_encoded.drop(columns=['examide', 'citoglipton','glimepiride-pioglitazone','metformin-rosiglitazone'],axis=1,inplace=True)

In [28]:
medics = ['metformin','repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin',
        'metformin-pioglitazone']
for i in medics:
    df_encoded.loc[df_encoded[i] == 'Up', [i]] = 3  
    df_encoded.loc[df_encoded[i] == 'Down', [i]] = 1 
    df_encoded.loc[df_encoded[i] == 'Steady', [i]] = 2 
    df_encoded.loc[df_encoded[i] == 'No', [i]] = 0
    df_test_encoded.loc[df_test_encoded[i] == 'Up', [i]] = 3  
    df_test_encoded.loc[df_test_encoded[i] == 'Down', [i]] = 1 
    df_test_encoded.loc[df_test_encoded[i] == 'Steady', [i]] = 2 
    df_test_encoded.loc[df_test_encoded[i] == 'No', [i]] = 0 
 


change and diabetesMed Process
1. n/a
2. binary map to 1/0

In [29]:
change_mapping = {'No':0,'Ch':1}
diabetesMed_mapping = {'No':0,'Yes':1}

df_encoded['change'] = df_encoded['change'].map(change_mapping)
df_encoded['diabetesMed'] = df_encoded['diabetesMed'].map(diabetesMed_mapping)

df_test_encoded['change'] = df_test_encoded['change'].map(change_mapping)
df_test_encoded['diabetesMed'] = df_test_encoded['diabetesMed'].map(diabetesMed_mapping)


readmitted Process
1.n/a
2. index mapping, No as 0, >30 as 1 and <30 as most significant as 2 and drop readmitted for test

In [30]:
readmitted_mapping = {'NO':0,'<30':2,'>30':1}

In [31]:

df_encoded['readmitted'] = df_encoded['readmitted'].map(readmitted_mapping)
df_test_encoded['readmitted'] = df_test_encoded['readmitted'].map(readmitted_mapping)

y_test_readmitted = df_encoded["readmitted"]
y_readmitted = df_test_encoded["readmitted"]

df_encoded.drop(columns=["readmitted"],axis=1,inplace=True)
df_test_encoded.drop(columns=["readmitted"],axis=1,inplace=True)


Normalization age, num_lab_procedures and num_medications to keep the same order of magnitude

In [36]:
from sklearn.preprocessing import MinMaxScaler


In [37]:
scaler = MinMaxScaler(feature_range=(0, 1))
features_to_scale = ["age","num_lab_procedures","num_medications"]
df_encoded[features_to_scale] = scaler.fit_transform(df_encoded[features_to_scale])
df_test_encoded[features_to_scale] = scaler.fit_transform(df_test_encoded[features_to_scale])


encode payer_code, medical_specialty
1. Find correlation internally with other feature in group of non-missing value
2. Use identified feature predict payer-code, medical_specialty
3. Prediction algorithm to be decide, could be KNN

In [30]:
# this part will reduce the dimension our training data

Dems Redct
1. PCA/PPCA
2. LDA/QDA
3. following to T-SNE
3. Autoencoders
4. Unsupervised Algorithmn

## Autoencoder

In [42]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

2024-01-12 01:02:01.149417: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-12 01:02:01.149474: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-12 01:02:01.150691: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-12 01:02:01.156240: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-12 01:02:02.200782: W tensorflow/compiler/tf2

In [43]:
# Number of features in your dataset
n_features = len(df_encoded.columns)  # Change this to the actual number of features

# Define the encoder
input_layer = Input(shape=(n_features,))
encoder = Dense(64, activation='relu')(input_layer)
encoder = Dense(32, activation='relu')(encoder)

# Define the bottleneck
bottleneck = Dense(10, activation='relu')(encoder)  # This is the latent space representation

# Define the decoder (mirror the encoder)
decoder = Dense(32, activation='relu')(bottleneck)
decoder = Dense(64, activation='relu')(decoder)

# Output layer
output_layer = Dense(n_features, activation='sigmoid')(decoder)  # Use 'sigmoid' or 'relu'

# Define the autoencoder model
autoencoder = Model(inputs=input_layer, outputs=output_layer)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mse')


In [ ]:
# Assuming you've split your DataFrame into training and validation sets
# X_train, X_val = ...
df_encoded = df_encoded.astype('float32')
df_test_encoded = df_test_encoded.astype('float32')
autoencoder.fit(df_encoded, df_encoded,  # The target is the input data itself
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(df_test_encoded, df_test_encoded))


In [46]:
encoder_model = Model(inputs=input_layer, outputs=bottleneck)
compressed_data = encoder_model.predict(df_encoded)
reconstructed_data = autoencoder.predict(df_encoded)


2793/2793 [==============================] - 2s 766us/step


In [48]:
reconstructed_data

array([[9.9854457e-01, 4.7584024e-01, 1.0000000e+00, ..., 8.6678155e-18,
        2.4750526e-04, 1.0569037e-30],
       [9.9737459e-01, 5.3435659e-01, 1.0000000e+00, ..., 4.8652789e-20,
        3.2783678e-01, 0.0000000e+00],
       [4.9697990e-03, 6.0271382e-01, 1.0000000e+00, ..., 1.4124907e-14,
        2.7947295e-02, 1.9507396e-31],
       ...,
       [3.1131611e-03, 5.7107723e-01, 1.0000000e+00, ..., 4.7571149e-14,
        8.2414329e-02, 2.6821391e-29],
       [9.9998325e-01, 6.2761122e-01, 1.0000000e+00, ..., 3.6573430e-24,
        8.0727950e-02, 0.0000000e+00],
       [5.3291032e-03, 6.3840473e-01, 1.0000000e+00, ..., 4.3646503e-10,
        2.9602233e-01, 4.3871168e-22]], dtype=float32)

In [ ]:
# from sklearn.preprocessing import StandardScaler, OneHotEncoder
# from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE
# import matplotlib.pyplot as plt


# # Simulating Data
# np.random.seed(0)
# num_samples = 1000
# num_features = 5

# # Numerical data
# numeric_data = np.random.randn(num_samples, num_features)

# # Categorical data (let's say, colors)
# colors = ['Red', 'Green', 'Blue']
# categorical_data = np.random.choice(colors, size=num_samples)

# # Convert categorical data to one-hot encoding
# encoder = OneHotEncoder(sparse=False)
# categorical_encoded = encoder.fit_transform(categorical_data.reshape(-1, 1))

# # Combining numerical and categorical data
# combined_data = np.hstack((numeric_data, categorical_encoded))

# # Standardize the numerical features
# scaler = StandardScaler()
# scaled_data = scaler.fit_transform(combined_data)

# # Apply PCA
# pca = PCA(n_components=0.95)  # Keep 95% of the variance
# pca_result = pca.fit_transform(scaled_data)

# # Apply t-SNE
# tsne = TSNE(n_components=2, perplexity=30, n_iter=300)
# tsne_result = tsne.fit_transform(pca_result)

# # Plotting the results
# sns.set(rc={'figure.figsize':(10,8)})
# sns.scatterplot(x=tsne_result[:,0], y=tsne_result[:,1], hue=categorical_data, palette='bright')
# plt.title('t-SNE plot of the dataset')
# plt.xlabel('t-SNE Axis 1')


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

input_shape = combined_data.shape[1]  # combined data from previous steps
encoding_dim = 32  # example of encoding dimension

# This is our input placeholder
input_data = Input(shape=(input_shape,))

# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_data)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(input_shape, activation='sigmoid')(encoded)

# This model maps an input to its reconstruction
autoencoder = Model(input_data, decoded)

autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(combined_data, combined_data, epochs=50, batch_size=256, shuffle=True)


# Model Building
We will build two models: 
1. A traditional machine learning model using Random Forest.
2. A deep learning model using PyTorch.


At the first step we will try to use the Random Forest method to get the result

In [ ]:
# the code for Random Forest algorithm

In [ ]:
# the code for Nerual Network

# Model Training


In [ ]:
# training loop
# Random Forest
X = df_encoded.drop('readmitted', axis=1)
Y = df_encoded['readmitted']
rf_classifier = HistGradientBoostingClassifier(max_iter=100, random_state=42)
rf_classifier.fit(X, Y)

# Nerual Network

# Model Evaluation

In [ ]:
# use testing dataset to predict
X_test = df_test_encoded.drop('readmitted', axis=1)
Y_test = df_test_encoded['readmitted']

Y_pred = rf_classifier.predict(X_test)
print(Y_pred)
print(Y_test)
print(accuracy_score(Y_test,Y_pred))